<a href="https://colab.research.google.com/github/ShangWeiKuo/106-2-R-project/blob/master/Code/ResearchMethod/health/3day/LSTM_ManytoOne_health_w3day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tsmoothie
!pip install keras-hypetune

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tsmoothie.smoother import *
from tsmoothie.utils_func import create_windows

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.optimizers import Adam
from kerashypetune import KerasGridSearch
from pandas import read_csv
from pandas import DataFrame
import numpy as np
import pickle

In [ ]:
window_shape = 3
time_h = 1

In [ ]:
import rpy2 
%load_ext rpy2.ipython

In [ ]:
%%R
require(readr)
require(dplyr)
df = read_csv("/content/drive/MyDrive/paper/raw_covid_data.csv")

R[write to console]: Loading required package: readr

R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





── Column specification ────────────────────────────────────────────────────────
cols(
  positive = col_double(),
  death = col_double(),
  new_test = col_double(),
  new_case = col_double(),
  dis_less_one = col_double(),
  dis_one_twtf = col_double(),
  dis_twtf_hum = col_double(),
  dis_higher_hum = col_double(),
  flight_d_volume = col_double(),
  flight_i_volume = col_double(),
  person_screened_number = col_double()
)



In [ ]:
%%R
install.packages("forecast"); library(forecast)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =


In [ ]:
%%R
ts_df <- matrix(1:4554, ncol = 11, nrow = 414)
for(i in c(1:ncol(df))){
  ts_df[, i] = ts(df[, i], frequency=1)
}
ts_df <- ts_df %>% data.frame()
train <- ts_df[1:331, 1:4]
test <- ts_df[332:414, 1:4]

colnames(train) <- colnames(df)[1:4]
colnames(test) <- colnames(df)[1:4]

cols = colnames(train)

In [ ]:
train=%R train
test=%R test
cols=%R cols

In [ ]:
train_y = pd.DataFrame(train['new_case'])
test_y = pd.DataFrame(test['new_case'])

In [ ]:
### CREATE TRAIN/VALIDATION WINDOWS W/ BOOTSTRAP ###

X_train = create_windows(train[cols], window_shape=window_shape, end_id=-time_h)
y_train = create_windows(train_y, window_shape=time_h, start_id=window_shape)

print(X_train.shape, y_train.shape)

(328, 3, 4) (328, 1, 1)


In [ ]:
### CREATE TEST WINDOWS ###

X_test = create_windows(test[cols], window_shape=window_shape, end_id=-time_h)
y_test = create_windows(test_y, window_shape=time_h, start_id=window_shape)

In [ ]:
Min_train = []
Max_train = []

In [ ]:
for i in range(X_train.shape[2]):
  Min_temp =  min(X_train[:, :, i].reshape(X_train[:, :, i].shape[0] * X_train[:, :, i].shape[1]))
  Min_train.append(Min_temp)
np.vstack(Min_train)

Min_train = np.array(Min_train)

In [ ]:
for i in range(X_train.shape[2]):
  Max_temp =  max(X_train[:, :, i].reshape(X_train[:, :, i].shape[0] * X_train[:, :, i].shape[1]))
  Max_train.append(Max_temp)
np.vstack(Max_train)

Max_train = np.array(Max_train)

In [ ]:
y_train = (y_train[...,[-1]] - Min_train[...,[-1]]) / (Max_train[...,[-1]] - Min_train[...,[-1]])
y_test = (y_test[...,[-1]] - Min_train[...,[-1]]) / (Max_train[...,[-1]] - Min_train[...,[-1]])

for i in range(X_train.shape[2]):
  if (Max_train[i] != 0): 
    X_train[...,[i]] = (X_train[...,[i]] - Min_train[i]) / (Max_train[i] - Min_train[i])
    X_test[...,[i]] = (X_test[...,[i]] - Min_train[i]) / (Max_train[i] - Min_train[i])

In [ ]:
def get_model_mse(param):
    
    inp = Input((window_shape,len(cols)))
    enc = LSTM(param['lstm_unit'], activation=param['lstm_act'], 
               return_sequences=False)(inp)
    x = RepeatVector(time_h)(enc)
    
    out = Dense(1)(x)
    
    model = Model(inp, out)
    model.compile(Adam(lr=param['lr']), 'mse')
    
    return model

In [ ]:
param_grid = {
    'lstm_unit': [128,64], 
    'lstm_act': ['tanh'],
    'lr': [1e-4,1e-3], 
    'epochs': 100, 
    'batch_size': [16,32,128]
}

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)
hypermodel_mse = get_model_mse
kgs_mse = KerasGridSearch(hypermodel_mse, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs_mse.search(X_train, y_train, validation_data=(X_test, y_test), callbacks=[es])


12 trials detected for ('lstm_unit', 'lstm_act', 'lr', 'epochs', 'batch_size')


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



***** (1/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 16})
Restoring model weights from the end of the best epoch.
Epoch 00068: early stopping
SCORE: 0.00237 at epoch 59

***** (2/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 32})
Restoring model weights from the end of the best epoch.
Epoch 00089: early stopping
SCORE: 0.00222 at epoch 89

***** (3/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 128})
SCORE: 0.01871 at epoch 100

***** (4/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 16})
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
SCORE: 0.00217 at epoch 3

***** (5/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 32})
Restoring model weights from the end of the best epoch.
Epoch 00017: ea

<kerashypetune.KerasGridSearch>

In [ ]:
pred_mse = kgs_mse.best_model.predict(X_test)
pred_mse

array([[[0.39097494]],

       [[0.38807032]],

       [[0.36550465]],

       [[0.32353237]],

       [[0.3078144 ]],

       [[0.30884394]],

       [[0.33137447]],

       [[0.33779073]],

       [[0.34683183]],

       [[0.32234675]],

       [[0.28652304]],

       [[0.25670552]],

       [[0.2685128 ]],

       [[0.28764224]],

       [[0.3121539 ]],

       [[0.30406904]],

       [[0.28034997]],

       [[0.23855096]],

       [[0.21295694]],

       [[0.21038848]],

       [[0.23215002]],

       [[0.24646273]],

       [[0.2592158 ]],

       [[0.24749675]],

       [[0.2245292 ]],

       [[0.20945299]],

       [[0.21811828]],

       [[0.2336435 ]],

       [[0.24451336]],

       [[0.23986271]],

       [[0.22621408]],

       [[0.2008588 ]],

       [[0.19245002]],

       [[0.19526261]],

       [[0.20700672]],

       [[0.21072221]],

       [[0.21830899]],

       [[0.20765756]],

       [[0.18647379]],

       [[0.16927722]],

       [[0.17321199]],

       [[0.18524

In [ ]:
pred_mse = np.squeeze(pred_mse, -1)

In [ ]:
pred_mse = ((Max_train[3] - Min_train[3]) * pred_mse) + Min_train[3]

In [ ]:
pd.DataFrame(pred_mse).to_csv("/content/drive/MyDrive/paper/health/PredictionData_ManytoOne_health_w3day_mse.csv")

In [ ]:
def get_model_mae(param):
    
    inp = Input((window_shape,len(cols)))
    enc = LSTM(param['lstm_unit'], activation=param['lstm_act'], 
               return_sequences=False)(inp)
    x = RepeatVector(time_h)(enc)
    
    out = Dense(1)(x)
    
    model = Model(inp, out)
    model.compile(Adam(lr=param['lr']), 'mae')
    
    return model

In [ ]:
hypermodel_mae = get_model_mae
kgs_mae = KerasGridSearch(hypermodel_mae, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs_mae.search(X_train, y_train, validation_data=(X_test, y_test), callbacks=[es])


12 trials detected for ('lstm_unit', 'lstm_act', 'lr', 'epochs', 'batch_size')

***** (1/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 16})


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Restoring model weights from the end of the best epoch.
Epoch 00058: early stopping
SCORE: 0.03654 at epoch 52

***** (2/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 32})
Restoring model weights from the end of the best epoch.
Epoch 00099: early stopping
SCORE: 0.03596 at epoch 92

***** (3/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 128})
SCORE: 0.11679 at epoch 100

***** (4/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 16})
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
SCORE: 0.04109 at epoch 3

***** (5/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 32})
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
SCORE: 0.03952 at epoch 6

***** (6/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0

<kerashypetune.KerasGridSearch>

In [ ]:
pred_mae = kgs_mae.best_model.predict(X_test)
pred_mae

array([[[0.3621031 ]],

       [[0.35717022]],

       [[0.3360115 ]],

       [[0.2899364 ]],

       [[0.27306128]],

       [[0.27937636]],

       [[0.30558443]],

       [[0.320425  ]],

       [[0.32562327]],

       [[0.29965425]],

       [[0.25770807]],

       [[0.23176098]],

       [[0.24775448]],

       [[0.26914468]],

       [[0.28319883]],

       [[0.27568942]],

       [[0.25444618]],

       [[0.21682453]],

       [[0.19263859]],

       [[0.19526976]],

       [[0.22005579]],

       [[0.23495069]],

       [[0.241795  ]],

       [[0.22853538]],

       [[0.2022267 ]],

       [[0.19301224]],

       [[0.21327779]],

       [[0.23775795]],

       [[0.2500049 ]],

       [[0.24472982]],

       [[0.22652632]],

       [[0.19426398]],

       [[0.18406156]],

       [[0.1983634 ]],

       [[0.22297794]],

       [[0.23250529]],

       [[0.23147166]],

       [[0.213453  ]],

       [[0.18322784]],

       [[0.17027438]],

       [[0.1857718 ]],

       [[0.20804

In [ ]:
pred_mae = np.squeeze(pred_mae, -1)

In [ ]:
pred_mae = ((Max_train[3] - Min_train[3]) * pred_mae) + Min_train[3]

In [ ]:
pd.DataFrame(pred_mae).to_csv("/content/drive/MyDrive/paper/health/PredictionData_ManytoOne_health_w3day_mae.csv")

In [ ]:
kgs_mae.best_model.save("/content/drive/MyDrive/paper/feature_importance/health/ManytoOne/w3day")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/paper/feature_importance/health/ManytoOne/w3day/assets


In [ ]:
# import pickle
with open('/content/drive/MyDrive/paper/feature_importance/health/ManytoOne/w3day/X_test_health_w3day.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('/content/drive/MyDrive/paper/feature_importance/health/ManytoOne/w3day/cols_health_w3day.pkl', 'wb') as f:
    pickle.dump(cols, f)